In [1]:
import requests
import json
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# web_scrap function web scraps from imdb to get imdbid number for each movie from imdb.com

In [3]:
def web_scrap():
    from bs4 import BeautifulSoup
    page = requests.get('https://www.imdb.com/chart/toptv/?sort=rk,asc&mode=simple&page=1')
    soup = BeautifulSoup(page.content, 'html.parser')
    movie_titles = soup.find_all(class_='titleColumn')
    code_title_dict = {mov.a['href'].split('/')[2]:mov.a.text for mov in movie_titles}
    return code_title_dict

In [4]:
code_title_dict = web_scrap()

In [ ]:
#seperating imdbid from movie title into a seperate list so we can run api key find with using the imdbid

In [5]:
def seperate_imdb_codes(code_title_dict):
    imdb_codes = []
    for k in code_title_dict.keys():
        imdb_codes.append('&i=' + k)
    return imdb_codes

In [6]:
imdb_codes = seperate_imdb_codes(code_title_dict)

In [7]:
api_key = '944239dd'

In [8]:
#omdb_api_call will take all the imdbid from the imdbid list and use that as a param in an api call at omdbapi.com
#and loop back the results from the calls

In [9]:
def omdb_api_call(imdb_codes, api_key):
    new_list = []
    for imdbid in imdb_codes:
        url = 'http://www.omdbapi.com/?apikey=' + api_key + imdbid
        response = requests.get(url)
        results = json.loads(response.text)
        new_list.append(results)
    return new_list

In [10]:
new_list = omdb_api_call(imdb_codes, api_key)

In [ ]:
#columns = ['imdbID','Title','Runtime','Genre','Awards','Meta','IMDb Rating','IMDb Votes','Seasons']
# columns we would like to see in the database

In [ ]:
#creating_df function will create a df from the collected data and clean up some of the columns.

In [14]:
def creating_df(scrapped_info):
    remove_columns = ['Year','Released','Director','Writer','Actors','Plot','Rated','Poster','Ratings','Response','Country','Language','Metascore']
    df = pd.DataFrame(scrapped_info)
    df.set_index('imdbID')
    df = df.drop(columns = remove_columns)
    return df

# creating_df function takes the scrapped_info(list) and creates a dataframe structure
# also takes out the unnecessary columns that will clog up our dataframe
# sets the index to the imdbID
# returns a dataframe with the columns we need

In [15]:
df = creating_df(new_list)